In [1]:
pip install mani_skill

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 MB 19.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 623.8/623.8 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 85.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 35.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.8/113.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.8/704.8 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 638.2/638.2 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.1 MB/s eta 0:00:00
  Attempting uni

In [2]:
import os
from glob import glob
import json
import h5py
from tqdm.notebook import tqdm
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

import gymnasium as gym
import mani_skill.envs
from mani_skill.utils.wrappers.flatten import FlattenRGBDObservationWrapper
from mani_skill.vector.wrappers.gymnasium import ManiSkillVectorEnv
from mani_skill.utils.registration import TimeLimitWrapper

/usr/local/lib/python3.10/dist-packages/sapien/__init__.py:2: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3121: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3121: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/l

In [3]:
def yield_mini_batches(data_folder):
    all_dirs = os.listdir(data_folder)
    random.shuffle(all_dirs)

    buffer_episode = None  

    for curr_dir in all_dirs:
        print(f"Processing: {curr_dir}")
        sub_dir_path = os.path.join(data_folder, curr_dir, curr_dir)
        
        h5_files = [f for f in os.listdir(sub_dir_path) if f.endswith(".h5")]
        random.shuffle(h5_files)

        for h5f in h5_files:
            h5_path = os.path.join(sub_dir_path, h5f)
            
            episode_generator = load_trajectories(h5_path)
            for episode in episode_generator:
                if buffer_episode is not None:
                    rgb = torch.cat([buffer_episode["rgb"], episode["rgb"]], dim=0)
                    acts = torch.cat([buffer_episode["actions"], episode["actions"]], dim=0)
                    buffer_episode = None 
                else:
                    buffer_episode = episode
                    continue

                mini_batch = {
                    "rgb": rgb,
                }
                mini_acts = acts
                yield mini_batch, mini_acts

                del mini_batch, mini_acts, rgb, state, acts

    if buffer_episode is not None:
        mini_batch = {
            "rgb": buffer_episode["rgb"],
        }
        mini_acts = buffer_episode["actions"]
        yield mini_batch, mini_acts

        del mini_batch, mini_acts, buffer_episode
        
def load_trajectories(h5_path):
    json_path = h5_path.replace(".h5", ".json")

    with open(json_path, "r", encoding="utf-8") as file:
        data = json.load(file)
    
    with h5py.File(h5_path, "r") as hf:
        for i, episode in enumerate(data["episodes"]):
            if episode["success"]:
                group_name = f"traj_{i}"             
                grp = hf[group_name]

                rgb_np = grp["obs"]["rgb"][:-1] 
                rgb_np = np.transpose(rgb_np, (0, 3, 1, 2)) 
                rgb_tensor = torch.from_numpy(rgb_np).float()

                acts_np = grp["actions"][:]
                acts_tensor = torch.from_numpy(acts_np).float()

                episode_data = {
                    "rgb": rgb_tensor,
                    "actions": acts_tensor
                }
                yield episode_data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
class Logger:
    def __init__(self, log_wandb=False):
        self.log_wandb = log_wandb
    def add_scalar(self, tag, scalar_value, step):
        if self.log_wandb:
            wandb.log({tag: scalar_value}, step=step)

In [6]:
class FeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        cnn = models.resnet18(pretrained=True)
        cnn.fc = nn.Identity()
        
        self.extractors = nn.Sequential(cnn, nn.ReLU())
        self.mlp = nn.Sequential(
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, 8),
            nn.Tanh()
        )
    
    def forward(self, observations):
        obs = observations["rgb"]
        if obs.shape[3] == 6:
           obs = obs.float().permute(0,3,1,2)
        obs = obs / 255
        feats_1 = extractor(obs[:, :3, :, :]) 
        feats_2 = extractor(obs[:, 3:, :, :])
        feats = torch.concat((feats_1, feats_2), dim=1)

        return self.mlp(feats)

In [7]:
def evaluate(model, eval_envs, logger, global_step):
    print(f"Evaluating at {global_step} steps")
    model.eval()
    eval_obs, _ = eval_envs.reset()
    eval_metrics = defaultdict(list)
    for _ in tqdm(range(80)):
        with torch.no_grad():
            eval_obs, eval_rew, eval_terminations, eval_truncations, eval_infos = eval_envs.step(model(eval_obs))
            if "final_info" in eval_infos:
                mask = eval_infos["_final_info"]
                for k, v in eval_infos["final_info"]["episode"].items():
                    eval_metrics[k].append(v)
    for k, v in eval_metrics.items():
        mean = torch.stack(v).float().mean()
        logger.add_scalar(f"eval/{k}", mean, global_step)
        print(f"eval_{k}_mean={mean}")

In [8]:
def train(
          data_folder,
          device,
          lr=1e-4,
          checkpoint_dir="checkpoints",
          eval_freq = 10
):
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("Running training")

    env_kwargs = dict(obs_mode="rgb", render_mode="all", sim_backend="gpu")
    eval_envs = gym.make("StackCube-v1", num_envs=100, **env_kwargs)
    eval_envs = TimeLimitWrapper(eval_envs, max_episode_steps=80)
    eval_envs = FlattenRGBDObservationWrapper(eval_envs, rgb=True, depth=False, state=True)
    eval_envs = ManiSkillVectorEnv(eval_envs, 100, ignore_terminations=True, record_metrics=True)
    
    wandb.login(key="")
    config = {}
    config["eval_env_cfg"] = dict(
                                  **env_kwargs, 
                                  num_envs=100, 
                                  env_id="StackCube-v1", 
                                  reward_mode="normalized_dense", 
                                  env_horizon=80
    )
    wandb.init(
                project="pretrain",
                sync_tensorboard=False,
                config=config,
                name="pretrain",
                save_code=True,
    )
    logger = Logger(log_wandb=True)
    
    model = FeatureExtractor().to(device)
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    
    global_step = 0
    for i, (obs_chunk, acts_chunk) in enumerate(yield_mini_batches(data_folder)):
        if i % eval_freq == 1:
            evaluate(model, eval_envs, logger, global_step)
            model.train()
        global_step += 160
        for _ in range(3):
            obs_chunk["rgb"] = obs_chunk["rgb"].to(device)
            obs_chunk["state"] = obs_chunk["state"].to(device)
            acts_chunk = acts_chunk.to(device)

            pred_actions = model(obs_chunk)  
            loss = criterion(pred_actions, acts_chunk)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        logger.add_scalar("train/loss", loss.item(), global_step)

        del obs_chunk, acts_chunk, pred_actions
        torch.cuda.empty_cache()

    ckpt_path = f"model.pth"
    torch.save({
        "epoch": 1,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict()
    }, ckpt_path)
    print(f"Checkpoint saved: {ckpt_path}")

    print("Training finished.")

In [ ]:
data_folder = "/kaggle/input/trajectories-dataset"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train(
    data_folder=data_folder, 
    device=device, 
    lr=1e-4, 
    checkpoint_dir="checkpoints", 
    eval_freq=50
)

Running training
Download complete.


/usr/local/lib/python3.10/dist-packages/torch/random.py:167: UserWarning: CUDA reports that you have 2 available devices, and you have used fork_rng without explicitly specifying which devices are being used. For safety, we initialize *every* CUDA device by default, which can be quite slow if you have a lot of CUDAs. If you know that you are only making use of a few CUDA devices, set the environment variable CUDA_VISIBLE_DEVICES or the 'devices' keyword argument of fork_rng with the set of devices you are actually using. For example, if you are using CPU only, set device.upper()_VISIBLE_DEVICES= or devices=[]; if you are using device 0 only, set CUDA_VISIBLE_DEVICES=0 or devices=[0].  To initialize all devices and suppress this warning, set the 'devices' keyword argument to `range(torch.cuda.device_count())`.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 149MB/s] 


Processing: traj_3
Evaluating at 160 steps


  0%|          | 0/80 [00:00<?, ?it/s]

...

eval_success_once_mean=0.0
eval_return_mean=18.621978759765625
eval_episode_len_mean=80.0
eval_reward_mean=0.2327747344970703
eval_success_at_end_mean=0.0
Evaluating at 40160 steps


  0%|          | 0/80 [00:00<?, ?it/s]

eval_success_once_mean=0.009999999776482582
eval_return_mean=19.915142059326172
eval_episode_len_mean=80.0
eval_reward_mean=0.24893926084041595
eval_success_at_end_mean=0.0
Processing: traj_16
Evaluating at 48160 steps


  0%|          | 0/80 [00:00<?, ?it/s]

eval_success_once_mean=0.03999999910593033
eval_return_mean=22.920148849487305
eval_episode_len_mean=80.0
eval_reward_mean=0.28650185465812683
eval_success_at_end_mean=0.0
